In [24]:
#-*- encoding: utf8 -*-

In [1]:
import requests
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import pandas as pd

In [66]:


## chromedriver 위치
driver = webdriver.Chrome("c:/git/chromedriver.exe")
driver.get("https://kyochon.revelup.com/login/?next=/dashboard/")

- 날짜 정보 준비

In [1]:
date_from= []
date_from.append(input('date from(dd): '))
date_from.append(input('month from(mm): '))
date_from.append(input('year from(yyyy): '))

range_from = date_from[0]+'%2F'+date_from[1]+'%2F'+date_from[2]+'+00%3A00%3A27'

date from(dd): 18
month from(mm): 12
year from(yyyy): 2018


In [2]:
date_to= []
date_to.append(input('date to(dd): '))
date_to.append(input('month to(mm): '))
date_to.append(input('year to(yyyy): '))

range_to = date_to[0]+'%2F'+date_to[1]+'%2F'+date_to[2]+'+00%3A00%3A27'

date to(dd): 24
month to(mm): 12
year to(yyyy): 2018


In [ ]:
#download_directory = input('where will your files be downloaded?: (한글 불가)')

In [58]:
# 날짜 정보 기반 리포트 링크
sales_summary = 'https://kyochon.revelup.com/brand/reports/sales_summary/json/?dining_option=&employee=&online_app=&online_app_type=&online_app_platform=&show_unpaid=1&show_irregular=1&range_from={}&range_to={}&format=csv'.format(range_from, range_to)
hourly_sales = 'https://kyochon.revelup.com/reports/hourly_sales/export/csv/?aggregate_format=hours&employee=&online_app=&online_app_type=&online_app_platform=&dining_option=&show_unpaid=1&show_irregular=1&no-filter=0&day_of_week=&range_from={}&range_to={}'.format(range_from, range_to)
product_mix = 'https://kyochon.revelup.com/reports/product_mix/data/?sort_by=&sort_reverse=&combo_expand=&employee=&online_app=&online_app_type=&online_app_platform=&dining_option=&show_unpaid=1&show_irregular=1&sort_view=1&show_class=1&quantity_settings=0&no-filter=0&day_of_week=&range_from={}&range_to={}&format=csv'.format(range_from, range_to)

- 로그인

In [67]:
driver.find_element_by_xpath('//*[@id="id_username"]').send_keys("JayceKim")
driver.find_element_by_xpath('//*[@id="id_password"]').send_keys("Qwer1234!!")
driver.find_element_by_xpath('//*[@id="form-login"]/fieldset/div[3]/input').click()
driver.implicitly_wait(3)

- 반복문으로 매장 선택

In [76]:
# 매장 팝업 열기
driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[2]/div[2]').click()

In [61]:
# 총 매장 수 호출
import re

tmp = driver.find_element_by_class_name('fancytree-title').text
n_store = int(re.findall(string=tmp, pattern = '[0-9]+')[0])
print(n_store)

11


In [80]:
import time

# 매장 별 페이지 호출, 자료 다운로드
for i in range(1, n_store+1):
    print('data fetching on store #{}...'.format(str(i)))
    if i > 1:
        driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[2]/div[2]').click()
        driver.implicitly_wait(5)
    driver.find_element_by_xpath('//*[@id="establishments-tree"]/div/div[3]/ul/li[1]/ul/li[{}]/span[2]/span[3]'.format(i)).click()
    time.sleep(5)
    
    # 날짜 조정
    driver.get(sales_summary)
    driver.implicitly_wait(5)
    driver.get(hourly_sales)
    driver.implicitly_wait(5)
    driver.get(product_mix)
    driver.implicitly_wait(10)
    

data fetching on store #1...
data fetching on store #2...
data fetching on store #3...
data fetching on store #4...
data fetching on store #5...
data fetching on store #6...
data fetching on store #7...
data fetching on store #8...
data fetching on store #9...
data fetching on store #10...
data fetching on store #11...


> 메뉴 업데이트 마다 확인 및 사용 (클래스 별로 메뉴 분류)

In [63]:
for i in stores:
    product_mix = "Product_Mix_{}_{}_00-00_{}_00-00.csv".format(i,
                                                            (date_from[2] + '-' +date_from[1] +'-'+ date_from[0]),
                                                            (date_to[2] + '-' +date_to[1] +'-'+ date_to[0]))
    if i =='arcoris':
        tmp = pd.read_csv('MY/'+product_mix).groupby('Class', as_index = False).sum()
    else:
        try:
            tmp = pd.concat([tmp, pd.read_csv('MY/'+product_mix).groupby('Class', as_index = False).sum()])
        except:
            pass
        
tmp = tmp.Class.unique()

In [68]:
tobe = ["Beverage", "Beverage", "Beverage", "Delete", "Delete",
 "Chicken", "Chicken", "Chicken", "Side", "Chicken",
 "Chicken", "Side", "Salad", "Beverage", "A-La-Carte",
 "Chicken", "Beverge", "Beverage", "Beverage", "Delete"]

menu = pd.DataFrame([tmp, tobe]).T
menu.columns = ['asis', 'tobe']
menu.to_csv("MY/menu.csv", index=False)

___

- 다운로드 기반 필요 데이터 추출

In [66]:
stores = ["arcoris", "ipc", "setia-city", "mytown", "genting", "pearl", "penang", "sunway-pyramid", "hap-seng", "pavilion", "1-utama"]
menu = pd.read_csv("MY/menu.csv")

> functions 정의

In [168]:
# 메뉴 별 판매비중 데이터 정제 함수 (메뉴 별 판매)

def pd_mix_func(data):
    global tmp_pd, sales_by_menu
    
    try:
        tmp_pd = pd.merge(pd.read_csv('MY/'+data).groupby("Class", as_index=False).sum(),
                   menu,
                   left_on="Class",
                   right_on = "asis").groupby('tobe').sum()['Total Sales']
    except:
        for i in range(len(tmp_pd)):
            tmp_pd[i] = NaN
    if store == stores[0]:
        sales_by_menu = pd.DataFrame(tmp_pd)
    else:
        sales_by_menu = pd.concat([sales_by_menu, pd.DataFrame(tmp_pd)], axis = 1, sort = False)
    

In [165]:
# 시간 별 판매 데이터 정제 (시간 별 판매)
def hourly_func(data):
    global sales_by_time
    
    temp = pd.read_csv('MY/'+data)[['Time', 'Sales']].iloc[9:24]
    
    if store == stores[0]:
        sales_by_time = pd.DataFrame(temp)
    else:
        sales_by_time = pd.concat([sales_by_time, pd.DataFrame(temp)["Sales"]], axis = 1, sort = False)


In [198]:
# 기초 데이터 정제 (매출 + 판매 타입)

def sales_func(data):
    global daily_gross_sales, daily_net_sales, sales_by_order
    
    
    temp = pd.read_csv('MY/'+data)#[['Gross Sales', 'Net Sales','Togo Sales', 'Eatin Sales', 'Delivery Sales',
                                   # 'Catering Sales', 'WebOrder', 'Online Sales', 'Shipping Sales', 'Takeaway']]
    temp1 = pd.DataFrame([temp[['Eatin Sales', 'Catering Sales']].sum().sum(),
                         temp[['Togo Sales', 'Takeaway']].sum().sum() ,
                         temp[['Delivery Sales', 'Online Sales', 'Shipping Sales']].sum().sum()],
                        index = ['Dining', 'Take-out', 'Delivery'])
    
    if store == stores[0]:
        daily_gross_sales = pd.DataFrame(temp['Gross Sales'])
        daily_net_sales = pd.DataFrame(temp['Net Sales'])
        sales_by_order = temp1
    else:
        daily_gross_sales = pd.concat([daily_gross_sales,temp['Gross Sales']], axis = 1, sort = False)
        daily_net_sales = pd.concat([daily_net_sales,temp['Net Sales']], axis = 1, sort = False)
        sales_by_order = pd.concat([sales_by_order, temp1], axis = 1, sort = False)
        


> 매장 별로 데이터 호출 및 정제 (위 함수 사용)

In [199]:
for store in stores:
    product_mix = "Product_Mix_{}_{}_00-00_{}_00-00.csv".format(store,
                                                            (date_from[2] + '-' +date_from[1] +'-'+ date_from[0]),
                                                            (date_to[2] + '-' +date_to[1] +'-'+ date_to[0]))
    hourly_sales = "Hourly_Sales_{}_{}_00-00_{}_00-00.csv".format(store,
                                                            (date_from[2] + '-' +date_from[1] +'-'+ date_from[0]),
                                                            (date_to[2] + '-' +date_to[1] +'-'+ date_to[0]))
    sales_summary = "Sales_Summary_{}_{}_00-00_{}_00-00.csv".format(store,
                                                            (date_from[2] + '-' +date_from[1] +'-'+ date_from[0]),
                                                            (date_to[2] + '-' +date_to[1] +'-'+ date_to[0]))
    
    print(store)
    pd_mix_func(product_mix)
    hourly_func(hourly_sales)
    sales_func(sales_summary)
    
    if store == stores[-1]:
        sales_by_time.columns = ['Time'] + stores
        sales_by_menu.columns = stores
        daily_gross_sales.columns = stores
        daily_net_sales.columns = stores
        sales_by_order.columns = stores

arcoris
ipc
setia-city
mytown
genting
pearl
penang
sunway-pyramid
hap-seng
pavilion
1-utama


In [ ]:
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

daily_net_sales.to_excel(writer, sheet_name="raw data")
daily_gross_sales.to_excel(writer, sheet_name="raw data", startrow=9)
sales_by_order.to_excel(writer, sheet_name="raw data", startrow=9+9)
sales_by_menu.to_excel(writer, sheet_name="raw data", startrow=9+9+5)
sales_by_time.to_excel(writer, sheet_name="raw data", startrow=9+9+5+9, index=False)

writer.save()

___
___
___

In [ ]:
pd.read_csv('c:/users/박찬우/downloads/Product_Mix_mytown_2018-12-18_00-00_2018-12-24_00-00.csv', engine='python')

___
___
___

In [201]:
daily_net_sales

,arcoris,ipc,setia-city,mytown,genting,pearl,penang,sunway-pyramid,hap-seng,pavilion,1-utama
0,5392.00,8260.83,13331.37,23412.00,16125.46,4044.21,9447.90,18892.50,0.0,27225.72,22679.82
1,4686.50,8546.08,14017.20,22913.39,14965.41,4761.20,7755.20,19487.41,0.0,28420.95,22468.19
2,5032.35,9647.50,15240.96,23411.68,17105.04,4846.10,11602.30,20625.25,0.0,30845.60,23362.63
3,6064.00,10087.55,15431.60,27002.30,18693.93,5418.40,12606.14,22769.90,0.0,32154.09,25716.99
4,6233.00,12336.93,18701.03,27333.83,25893.60,5909.10,16626.70,26467.44,0.0,35517.35,28620.30
5,4799.00,12734.66,20443.14,29827.16,26726.78,6804.00,17030.44,25851.60,0.0,35442.74,29713.54
6,32206.85,61613.55,97165.30,153900.36,119510.22,31783.01,75068.68,134094.10,0.0,189606.45,152561.47


In [237]:
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

In [238]:
daily_net_sales.to_excel(writer, sheet_name="raw data")
daily_gross_sales.to_excel(writer, sheet_name="raw data", startrow=9)
sales_by_order.to_excel(writer, sheet_name="raw data", startrow=9+9)
sales_by_menu.to_excel(writer, sheet_name="raw data", startrow=9+9+5)
sales_by_time.to_excel(writer, sheet_name="raw data", startrow=9+9+5+9, index=False)

In [221]:
wb = openpyxl.Workbook()

In [239]:
writer.save()

In [231]:
print(daily_net_sales.shape)
print(daily_gross_sales.shape)
print(sales_by_order.shape)
print(sales_by_menu.shape)
print(sales_by_time.shape)

(7, 11)
(7, 11)
(3, 11)
(7, 11)
(15, 12)


In [251]:
os.listdir(u"C:\\")

['$Recycle.Bin',
 'Addresses_in_the_City_of_Los_Angeles.csv',
 'bcl.log',
 'bootTel.dat',
 'DiskLock',
 'Documents and Settings',
 'DuzonDass',
 'Git',
 'hiberfil.sys',
 'InjectIntoProcess crash',
 'Intel',
 'mp',
 'MSOCache',
 'NeoICube',
 'pagefile.sys',
 'Pavilion.html',
 'PerfLogs',
 'Product_Mix_pavilion_2018-12-18_00-00_2018-12-24_00-00.csv',
 'Program Files',
 'Program Files (x86)',
 'ProgramData',
 'Programming',
 'Recovery',
 'Riot Games',
 'Shopping',
 'snf_log_20180529.txt',
 'stream_twitter.py',
 'swapfile.sys',
 'System Volume Information',
 'Temp',
 'tempWw.log',
 'tweet_kyochon.txt',
 'Users',
 'WaterwallSecurityDriveEnterpriseFV.wwsd',
 'Weekly Sales Report(MY).xlsx',
 'Windows',
 'WWCNT',
 'WWNTUSER',
 'XecureSSL',
 '_exception_log.txt']

In [226]:
for r in dataframe_to_rows(daily_net_sales, index = True, header = True):
    ws.append(r)

In [229]:
!pip install xlsxwriter


In [267]:
path.encode('utf-8')[3:].decode('utf-8')

'c:/Users/박찬우/Downloads/Hourly_Sales_arcoris_2018-12-18_00-00_2018-12-24_00-00.csv'

In [4]:
path = u'‪c:/Users/박찬우/Downloads/Hourly_Sales_arcoris_2018-12-18_00-00_2018-12-24_00-00.csv'
path.encode('utf-8')[3:]

b'c:/Users/\xeb\xb0\x95\xec\xb0\xac\xec\x9a\xb0/Downloads/Hourly_Sales_arcoris_2018-12-18_00-00_2018-12-24_00-00.csv'

In [7]:

f = open(u'‪c:/users/박찬우/downloads/hourly_sales_arcoris_2018-12-18_00-00_2018-12-24_00-00.csv', 'rb')

csv_f = csv.reader(f)

for row in csv_f:
    print(row)
f.close()

OSError: [Errno 22] Invalid argument: '\u202ac:/Users/박찬우/Downloads/Hourly_Sales_arcoris_2018-12-18_00-00_2018-12-24_00-00.csv'

In [27]:
'c:/users/박찬우/downloads/' + os.listdir('c:/users/박찬우/downloads')[185]

'c:/users/박찬우/downloads/Hourly_Sales_genting_2018-12-18_00-00_2018-12-24_00-00.csv'

In [33]:
pd.read_csv('c:/users/박찬우/downloads/Product_Mix_mytown_2018-12-18_00-00_2018-12-24_00-00.csv', engine='python')

,Row Type,Parent Product Class,Class,Name,Product Category,Product Subcategory,Parent Product,Qty,Weight,Total Sales,COGS
0,Product,Food,Ala Carte,Bibimbap (Soy),Ala Carte,Ala Carte,NaN,8,NaN,168.0,24.00
1,Product,Food,Ala Carte,Bibimbap (Spicy),Ala Carte,Ala Carte,NaN,16,NaN,336.0,48.00
2,Product,Food,Ala Carte,Dakganjeong,Ala Carte,Ala Carte,NaN,17,NaN,510.0,184.79
3,Product,Food,Ala Carte,Spicy Bulgogi,Ala Carte,Ala Carte,NaN,27,NaN,702.0,337.88
4,Product,Drinks,Bottled Water,Spritzer 600ml,Drinks,Bottled Water,NaN,463,NaN,1852.0,392.21
5,Product,Drinks,Bottled Water,Spritzer 600ml,Takeaway Drinks,Drinks,NaN,8,NaN,24.0,6.78
6,Product,Food,Combo,Clear Soup,Combo,* Upsell Combo Items,NaN,2803,NaN,0.0,1605.00
7,Product,Food,Combo,Iced Lemon Tea - Combo,Combo,* Upsell Combo Items,NaN,1305,NaN,0.0,4.18
8,Product,Food,Honey,1/2 Chicken - Honey (NC),Series,Honey,1/2 Chicken Honey,21,NaN,630.0,155.78
9,Product,Food,Honey,1/2 Chicken - Honey~,KyoChon Series,Honey,NaN,5,NaN,150.0,37.09


In [26]:
pd.read_csv('c:/users/박찬우/downloads/' + os.listdir('c:/users/박찬우/downloads')[185])

OSError: Initializing from file failed

In [ ]:
df1.to_excel(writer, sheet_name='Sheet1')  # Default position, cell A1.
df2.to_excel(writer, sheet_name='Sheet1', startcol=3)
df3.to_excel(writer, sheet_name='Sheet1', startrow=6)